In [1]:
# import
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import warnings
import streamlit as st
from keras.layers import Dense, Activation
from keras.models import Sequential
from tensorflow import keras
from sklearn.cluster import KMeans


# NLP
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import string

# this line tells jupyter notebook to put the plots in the notebook rather than saving them to file
%matplotlib inline

# this line makes plots prettier on mac retina screens - if you don't have one it shouldn't do anything
%config InlineBackend.figure_format = 'retina'


# Style
sns.set_style(style = 'darkgrid')

In [25]:
df = pd.read_csv("global/glassdoor_webscraped.csv")
df2 = pd.read_csv('japan/review_jp_nlp.csv')


In [26]:
df2.head()

,Unnamed: 0,reviewer_info,title,rating,review,date,review_translated,reviwe_translated_original,sentiment,emotion,em_x_senti
0,0,男性/ 営業/ 現職（回答時）/ 中途入社/ 在籍3年未満/ 正社員,Work style (working hours/holidays/systems),4.7,勤務時間・休日休暇:ワークライフバランスをとるかどうかは自分次第。ベースとなる勤務時間は9：...,2022年03月 6日,"[ ' Working ' , ' HoursHolidaysHolidays ' , ' ...",Working Hours/Holidays/Holidays: It's up to yo...,neg,joy,"('joy', 'neg')"
1,1,男性/ 営業/ 現職（回答時）/ 中途入社/ 在籍3年未満/ 正社員,Work style (working hours/holidays/systems),4.7,勤務時間・休日休暇:ワークライフバランスを保てるかどうかは自分次第。強制的に残業や休日の仕事...,2021年02月23日,"[ ' Working ' , ' HoursHolidaysHolidays ' , ' ...",Working Hours/Holidays/Holidays: It's up to yo...,pos,joy,"('joy', 'pos')"
2,2,男性/ Solution Engineer/ 現職（回答時）/ 中途入社/ 在籍3～5年/ ...,Annual salary/salary,5.0,給与制度:提示される想定年収の8割が固定給で残り2割がインセンティブインセンティブは四半期ご...,2020年11月20日,"[ ' Salary ' , ' system ' , ' 80 ' , ' ' , ' '...",Salary system: 80% of the estimated annual inc...,neg,joy,"('joy', 'neg')"
3,3,男性/ 6903/ 現職（回答時）/ 中途入社/ 在籍6～10年/ 正社員,Corporate culture/organizational system,4.9,企業カルチャー・社風:カスタマーサクセスを打ち出しており、導入後も運用をサポートする部門が支...,2021年04月 8日,"[ ' Corporate ' , ' culturecorporate ' , ' cul...",Corporate culture/corporate culture: The compa...,pos,joy,"('joy', 'pos')"
4,4,男性/ カスタマーサクセス/ 現職（回答時）/ 中途入社/ 在籍11～15年/ 正社員/ デ...,Benefits/office environment,5.0,福利厚生:オフィス内のカフェテリアは充実している。エスプレッソマシンは自動であるが、豆もそれ...,2020年06月 2日,"[ ' Welfare ' , ' ' , ' ' , ' ' , ' good ' , '...",Welfare: There is a good cafeteria in the offi...,neg,joy,"('joy', 'neg')"


In [21]:
df = df.loc[:, 'rating':'cons']
df2 = df2.loc[:, 'rating':'reviwe_translated_original']

In [20]:
df2.columns

Index(['Unnamed: 0', 'reviewer_info', 'title', 'rating', 'review', 'date',
       'review_translated', 'reviwe_translated_original'],
      dtype='object')